In [1]:
from games.kuhn.kuhn import KuhnPoker
from agents.counterfactualregret import CounterFactualRegret
from collections import OrderedDict

In [2]:
g = KuhnPoker()

In [6]:
g.reset()


In [30]:
g.reset()
g._hand

array([0, 2])

In [3]:
agent_classes = [ CounterFactualRegret, CounterFactualRegret ]
my_agents = {}
g.reset()
for i, agent in enumerate(g.agents):
    my_agents[agent] = agent_classes[i](game=g, agent=agent)

In [4]:
for agent in g.agents:
    print('Training agent ' + agent)
    my_agents[agent].train(100000)
    print('Agent ' + agent + ' policies:')
    print(OrderedDict(map(lambda n: (n, my_agents[agent].node_dict[n].policy()), sorted(my_agents[agent].node_dict.keys()))))
    print('')

Training agent agent_0
Agent agent_0 policies:
OrderedDict([('0', array([9.99969948e-01, 3.00516889e-05])), ('0b', array([9.99970069e-01, 2.99311583e-05])), ('0p', array([9.99970069e-01, 2.99311583e-05])), ('0pb', array([9.99977461e-01, 2.25391053e-05])), ('1', array([9.99970140e-01, 2.98596596e-05])), ('1b', array([0.5, 0.5])), ('1p', array([9.99969861e-01, 3.01386377e-05])), ('1pb', array([0.5, 0.5])), ('2', array([9.99969913e-01, 3.00869513e-05])), ('2b', array([0.5, 0.5])), ('2p', array([9.99970072e-01, 2.99284710e-05])), ('2pb', array([2.25655529e-05, 9.99977434e-01]))])

Training agent agent_1
Agent agent_1 policies:
OrderedDict([('0', array([9.99970351e-01, 2.96489564e-05])), ('0b', array([9.99969967e-01, 3.00327357e-05])), ('0p', array([9.99969967e-01, 3.00327357e-05])), ('0pb', array([9.99977763e-01, 2.22370469e-05])), ('1', array([9.99969936e-01, 3.00643377e-05])), ('1b', array([0.5, 0.5])), ('1p', array([9.99970045e-01, 2.99553665e-05])), ('1pb', array([0.5, 0.5])), ('2', ar

In [5]:
cum_rewards = dict(map(lambda agent: (agent, 0.), g.agents))
niter = 2000
for _ in range(niter):
    g.reset()
    turn = 0
    while not g.done():
        #print('Turn: ', turn)
        #print('\tPlayer: ', g.agent_selection)
        #print('\tObservation: ', g.observe(g.agent_selection))
        a = my_agents[g.agent_selection].action()
        #print('\tAction: ', g._moves[a])
        g.step(action=a)
        turn += 1
    #print('Rewards: ', g.rewards)
    for agent in g.agents:
        cum_rewards[agent] += g.rewards[agent]
print('Average rewards:', dict(map(lambda agent: (agent, cum_rewards[agent]/niter), g.agents)))


Average rewards: {'agent_0': 0.017, 'agent_1': -0.017}


In [51]:
print('Check learned policies against theoretical policies:')

Check learned policies against theoretical policies:


In [9]:
JX_b = my_agents[g.agents[0]].node_dict['0'].policy()[1]
print(f'Agent: 0 - Hand: J_ - History: [] - Probability of betting: {JX_b}')


Agent: 0 - Hand: J_ - History: [] - Probability of betting: 0.11362305231804722


In [10]:
QX_pb_b = my_agents[g.agents[0]].node_dict['1pb'].policy()[1]
print(f'Agent: 0 - Hand: Q_ - History: pb - Probability of betting: {QX_pb_b} - Theoretic value: {JX_b+1/3} -  Difference: {abs(QX_pb_b - (JX_b+1/3))}')


Agent: 0 - Hand: Q_ - History: pb - Probability of betting: 0.43739620157029147 - Theoretic value: 0.4469563856513805 -  Difference: 0.009560184081089051


In [12]:
KX_b = my_agents[g.agents[0]].node_dict['2'].policy()[1]
print(f'Agent: 0 - Hand: K_ - History: [] - Probability of betting: {KX_b} - Theoretic value: {3 * JX_b} -  Difference: {abs(KX_b - 3 * JX_b)}')


Agent: 0 - Hand: K_ - History: [] - Probability of betting: 0.33221948785943334 - Theoretic value: 0.34086915695414166 -  Difference: 0.008649669094708323


In [13]:
XJ_p_b = my_agents[g.agents[0]].node_dict['0p'].policy()[1]
print(f'Agent: 0 - Hand: _J - History: p - Probability of betting: {XJ_p_b} - Theoretic value: {1/3} -  Difference: {abs(XJ_p_b - 1/3)}')

Agent: 0 - Hand: _J - History: p - Probability of betting: 0.3359600141817036 - Theoretic value: 0.3333333333333333 -  Difference: 0.002626680848370311


In [7]:
XQ_b_b = my_agents[g.agents[0]].node_dict['1b'].policy()[1]
print(f'Agent: 0 - Hand: _Q - History: b - Probability of betting: {XQ_b_b} - Theoretic value: {1/3} -  Difference: {abs(XQ_b_b - 1/3)}')

Agent: 0 - Hand: _Q - History: b - Probability of betting: 0.5 - Theoretic value: 0.3333333333333333 -  Difference: 0.16666666666666669
